In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud,STOPWORDS



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


I will continue. 

In [ ]:
df = pd.read_csv('/kaggle/input/clinton-trump-tweets/tweets.csv')
df.head()

Let's check the data types

In [ ]:
df.dtypes

Let's check the null values

In [ ]:
df.isnull().sum() / df.shape[0]

There are some columns such that almost all of the values are missing. 

### LANGUAGES IN TWEETS

In [ ]:
df.groupby('lang')[['id']].count()

Most of the tweets are in english. There are some *undefined* tweets. Let's check some spanish tweets.

In [ ]:
df['original_author'] = df['original_author'].fillna("")

In [ ]:
df[(df['lang'] == 'es') & (df['original_author'] == '')][['lang', 'is_retweet', 'handle', 'text']].head()

### WHO ARE THEY RETWEETING

In [ ]:
retweeted = df[df['original_author'] != ''].groupby('original_author')[['text']].count().reset_index()
r_sorted = retweeted[retweeted['text'] >= 5].sort_values(by='text', ascending=False)
r_sorted


In [ ]:
plt.figure(figsize=(20, 10))

sns.barplot(x="text", y="original_author", data=r_sorted,
            label="Total", color="b")

plt.xlabel('Author')
plt.ylabel('Count')
plt.show()

We can check who is *TheBriefing2016*. When ı check I saw it was Hillary Clinton campaign account. You can check it fro  [this twitter link](https://twitter.com/thebriefing2016). And the HFA stands for Hillary for America. So the second account is also a Hillary Clinton campaign account.

Let's determine a column which shows the author of the tweet.

In [ ]:
def show_author(row):
    if row['original_author'] == "":
        return row['handle']
    return row['original_author']

df['author'] = df.apply(show_author, axis=1)

df[['author', 'handle', 'text']].head(20)

### LET'S CREATE THE CORPUS OBJECTS FOR BOTH HILLARY AND TRUMP

In [ ]:
df['text'] = df['text'].str.replace(r'[\n]', '')
df['text'] = df['text'].str.replace(r'&amp', '')
df['text'] = df['text'].str.replace(r'https?:\/\/.*[\r\n]*', '')
df['text']

In [ ]:
trump_tweets = df[(df['is_retweet']==False) & (df['handle']=="realDonaldTrump")]
clinton_tweets = df[(df['is_retweet']==False) & (df['handle']=="HillaryClinton")]

trump_tweets

In [ ]:
trump_corpus = trump_tweets[['text']]
clinton_corpus = clinton_tweets[['text']]
trump_corpus

In [ ]:
import nltk
from nltk.corpus import stopwords
STOP_WORDS = list(stopwords.words('english'))

def clean_corpus(df):
    

    df['text'] = df['text'].str.lower()
    df['text'] = df['text'].str.replace('\d+', '')
    df['text'] = df['text'].str.strip()

    punctuation_signs = list(")(?:!.,;@")

    for punct_sign in punctuation_signs:
        df['text'] = df['text'].str.replace(punct_sign, ' ')
    
    for stop_word in STOP_WORDS:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['text'] = df['text'].str.replace(regex_stopword, '')
        
    return df
    
trump_corpus = clean_corpus(trump_corpus)
clinton_corpus = clean_corpus(clinton_corpus)


In [ ]:
trump_corpus

### TRUMP CORPUS

In [ ]:
plt.figure(figsize = (20,20)) # Text that is not Fake
wc = WordCloud(max_words = 100 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(trump_corpus.text))
plt.imshow(wc , interpolation = 'bilinear')

### CLINTON CORPUS

In [ ]:
plt.figure(figsize = (20,20)) # Text that is not Fake
wc = WordCloud(max_words = 100 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(clinton_corpus.text))
plt.imshow(wc , interpolation = 'bilinear')